<a href="https://colab.research.google.com/github/gerbeldo/2022-dma-final/blob/main/petfinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [22]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier, plot_metric

%matplotlib inline
pd.set_option('display.max_columns', 30)
plt.rcParams['figure.figsize'] = [12.0, 8.0]

In [23]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Carga de datos

In [24]:
data = pd.read_csv("https://github.com/gerbeldo/2022-dma-final/blob/main/train.csv.zip?raw=true", compression = "zip")

# haciendo la acomodacion

## tipos

In [25]:
data = data.astype({'PhotoAmt' : int})

## ceros

Contamos los ceros y calculamos la proporción en cada columna. En algunas variables, como las `Breed` o `Color` representan valores faltantes. Las variables `Breed2` y `Color3` tienen alta proporción de faltantes, por lo que las eliminamos. La variable `Fee` también tiene muy alta proporción de ceros, pero no son faltantes. Lo mismo con la variable `VideoAmt`.

In [26]:
def count_zeros(array):
    n_zeros = sum(array == 0)
    prop_zeros = round(n_zeros / len(array), 2)
    return [n_zeros, prop_zeros]

(data
 .select_dtypes(include = "int64")
 .apply(count_zeros)
 .transpose()
 )

,0,1
Type,0.0,0.00
Age,179.0,0.01
Breed1,5.0,0.00
Breed2,10762.0,0.72
Gender,0.0,0.00
Color1,0.0,0.00
Color2,4471.0,0.30
Color3,10604.0,0.71
MaturitySize,0.0,0.00
FurLength,0.0,0.00


In [27]:
data.drop(["Breed2", "Color3"], axis = 1, inplace = True)

## Categóricas

Reemplazamos la variable `Name` por una nueva variable numerica `Has_name`, que toma valor 1 cuando la mascota tiene nombre y 0 en caso contrario.

In [30]:
data['Name'] = data['Name'].fillna('Unnamed')
data['Has_name'] = 1
data.loc[data['Name'] == 'Unnamed', 'Has_name'] = 0
data.drop("Name", axis = 1, inplace = True)

Eliminamos la variable `Description` para un primer modelo. Además, eliminamos la variable `PetID` ya que es inutil para la predicción.

In [31]:
data.drop(["Description", "PetID"], axis = 1, inplace = True)

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14993 entries, 0 to 14992
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Type           14993 non-null  int64 
 1   Age            14993 non-null  int64 
 2   Breed1         14993 non-null  int64 
 3   Gender         14993 non-null  int64 
 4   Color1         14993 non-null  int64 
 5   Color2         14993 non-null  int64 
 6   MaturitySize   14993 non-null  int64 
 7   FurLength      14993 non-null  int64 
 8   Vaccinated     14993 non-null  int64 
 9   Dewormed       14993 non-null  int64 
 10  Sterilized     14993 non-null  int64 
 11  Health         14993 non-null  int64 
 12  Quantity       14993 non-null  int64 
 13  Fee            14993 non-null  int64 
 14  State          14993 non-null  int64 
 15  RescuerID      14993 non-null  object
 16  VideoAmt       14993 non-null  int64 
 17  PhotoAmt       14993 non-null  int64 
 18  AdoptionSpeed  14993 non-n

# Escala/Normalización

Para xgboost no es necesario escalar/normalizar, pero si necesitáramos, serian éstas.

In [ ]:
#norm_vars = ["Age", "Quantity", "Fee", "VideoAmt", "PhotoAmt"]

# Train/Test split

In [33]:
x_train, x_test, y_train, y_test = train_test_split(data.drop("AdoptionSpeed", axis = 1),
                                                    data["AdoptionSpeed"],
                                                    test_size = 0.15,
                                                    random_state = 42)

# K-fold CV

Usamos 10 fold CV en la data de training para entrenar el modelo y optimizar hiperparametros.